In [12]:
import pandas as pd
import csv
from telethon import TelegramClient
import csv
import os
from dotenv import load_dotenv

In [24]:
# Load environment variables once
load_dotenv('.env')
api_id ='26559639'
api_hash ='3299ed6b2b0cbd2243470f18e1a77272'
phone = '+251984956023'
client = TelegramClient(phone, api_id, api_hash)
await client.start()

Signed in successfully as Eyoatam Tesfaye; remember to not break the ToS or you will risk an account ban!


In [42]:
# Function to scrape data from a single channel
async def scrape_channel(client, channel_username, writer, media_dir):
    entity = await client.get_entity(channel_username)
    channel_title = entity.title  # Extract the channel's title
    async for message in client.iter_messages(entity, limit=10000):
        media_path = None
        if message.media and hasattr(message.media, 'photo'):
            # Create a unique filename for the photo
            filename = f"{channel_username}_{message.id}.jpg"
            media_path = os.path.join(media_dir, filename)
            # Download the media to the specified directory if it's a photo
            await client.download_media(message.media, media_path)
        
        # Write the channel title along with other data
        writer.writerow([channel_title, channel_username, message.id, message.message, message.date, media_path])

 

async def main():
    await client.start()
    
    # Create a directory for media files
    media_dir = 'photos'
    os.makedirs(media_dir, exist_ok=True)

    # Open the CSV file and prepare the writer
    with open('../Data/telegram_data.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Channel Title', 'Channel Username', 'ID', 'Message', 'Date', 'Media Path'])  # Include channel title in the header
        
        # List of channels to scrape
        channels = [
            '@helloomarketethiopia',  # Existing channel
                 # You can add more channels here
            
        ]
        
        # Iterate over channels and scrape data into the single CSV file
        for channel in channels:
            await scrape_channel(client, channel, writer, media_dir)
            print(f"Scraped data from {channel}")
 
# async with client:
#     client.loop.run_until_complete(main())

In [45]:
file_path='..Data/telegram_data.csv'
df=pd.read_csv('file_path')

FileNotFoundError: [Errno 2] No such file or directory: 'file_path'

In [11]:
# Step 2: Check if the DataFrame loaded successfully
if df is not None:
    # Verify the presence of the "Message" column
    if 'Message' in df.columns:
        # Step 3: Define the function to convert messages to CoNLL format
        def to_conll_format(messages):
            conll_lines = []
            for message in messages:
                tokens = message.split()  # Split message into tokens
                for token in tokens:
                    # Implement your NER logic here
                    if "ዋጋ" in token:
                        label = "PRICE"
                    elif "ምርት" in token:
                        label = "PRODUCT"
                    elif "አውራጃ" in token:
                        label = "LOCATION"
                    else:
                        label = "O"  # Outside any entity
                    conll_lines.append(f"{token}\t{label}")
                conll_lines.append("")  # Blank line to separate messages
            return "\n".join(conll_lines)

        # Step 4: Convert the "Message" column to CoNLL format
        conll_output = to_conll_format(df['Message'])

        # Step 5: Print the CoNLL formatted output
        print(conll_output)
    else:
        print("The 'Message' column does not exist in the DataFrame.")
else:
    print("DataFrame could not be loaded.")

The 'Message' column does not exist in the DataFrame.


In [10]:

# Step 3: Convert the "Message" column to CoNLL format
if 'Message' in df.columns:
    conll_output = to_conll_format(df['Message'])
else:
    print("The 'Message' column does not exist in the DataFrame.")

# Step 4: Print the CoNLL formatted output
print(conll_output)

The 'Message' column does not exist in the DataFrame.


NameError: name 'conll_output' is not defined

In [ ]:
# Check if the "Message" column exists
if 'Message' not in df.columns:
    raise ValueError("The 'Message' column is missing from the dataset.")

# Step 2: Define the function to label entities
def label_entities(message):
    tokens = message.split()
    labeled_tokens = []

    for token in tokens:
        if "ዋጋ" in token or "ብር" in token:  # Simple logic for prices
            label = "B-PRICE" if "ዋጋ" in token else "I-PRICE"
        elif any(loc in token for loc in ["Addis", "Bole", "አዲስ", "ቦሌ"]):  # Example locations
            label = "B-LOC" if token in ["Addis", "አዲስ"] else "I-LOC"
        elif "የ" in token or "ነው" in token:  # Example products
            label = "B-PRODUCT" if "የ" in token else "I-PRODUCT"
        else:
            label = "O"  # Outside any entity

        labeled_tokens.append(f"{token}\t{label}")

    return "\n".join(labeled_tokens)

# Step 3: Process the messages and label entities
output_lines = []
for message in df['Message'].head(50):  # Adjust the number of messages to label
    labeled_message = label_entities(message)
    output_lines.append(labeled_message)
    output_lines.append("")  # Blank line to separate messages

# Step 4: Save the output to a CoNLL format text file
with open('labeled_data.conll', 'w', encoding='utf-8') as f:
    f.write("\n".join(output_lines))

print("Labeled data saved to 'labeled_data.conll'")